In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def grad_hk(x_k0: float, y_k0: float, x_k: float, y_k: float):
    return np.array(
        [
            [-(y_k - y_k0) / ((x_k - x_k0) ** 2 + (y_k - y_k0) ** 2)],
            [(x_k - x_k0) / ((x_k - x_k0) ** 2 + (y_k - y_k0) ** 2)],
            [0],
            [0],
        ]
    )


def update_fisher(
    p_0: np.ndarray,
    delta_t: float,
    n_before_turn: int,
    n_tot: int,
    phi: float,
    v_0: float,
    sigma: float,
    x_init: np.ndarray,
):
    speed_x = x_init[2, 0]
    speed_y = x_init[3, 0]
    pos_x_init = x_init[0, 0]
    pos_y_init = x_init[1, 0]
    speed_x_0_plus = v_0 * np.cos(phi)
    speed_y_0_plus = v_0 * np.sin(phi)
    big_phi = np.array(
        [[1, 0, delta_t, 0], [0, 1, 0, delta_t], [0, 0, 1, 0], [0, 0, 0, 1]]
    )
    inv_big_phi = np.linalg.inv(big_phi)
    fisher: np.ndarray = np.empty(shape=(n_tot, *(p_0.shape)))
    fisher[0] = np.linalg.inv(p_0)
    for j in range(n_tot - 1):
        if j + 1 <= n_before_turn:
            pos_x_0 = (j + 1) * v_0 * delta_t
            pos_y_0 = 0
        else:
            pos_x_0 = (
                n_before_turn * v_0 * delta_t
                + (j + 1 - n_before_turn) * speed_x_0_plus * delta_t
            )
            pos_y_0 = (j + 1 - n_before_turn) * speed_y_0_plus * delta_t
        pos_x = pos_x_init + (j + 1) * speed_x
        pos_y = pos_y_init + (j + 1) * speed_y
        evaluated_grad = grad_hk(x_k0=pos_x_0, y_k0=pos_y_0, x_k=pos_x, y_k=pos_y)
        fisher[j + 1] = (
            1 / sigma**2
        ) * evaluated_grad @ evaluated_grad.T + inv_big_phi.T @ fisher[j] @ inv_big_phi
    return fisher

In [ ]:
DELTA_T = 1
N_TOT = 100
N_BEFORE_TURN = 50

PHI_EMETTEUR = -20 * np.pi / 180
PHI_OBSERVATEUR = 135 * np.pi / 180
V_EMETT = 5
V_OBS = 10

x_emetteur_init = np.array(
    [[2000], [2000], [V_EMETT * np.cos(PHI_EMETTEUR)], [V_EMETT * np.sin(PHI_EMETTEUR)]]
)

p_0 = np.diag([1000**2, 1000**2, 10**2, 10**2])
sigma = 1 * np.pi / 180

fisher = update_fisher(
    p_0=p_0,
    delta_t=DELTA_T,
    n_before_turn=N_BEFORE_TURN,
    n_tot=N_TOT,
    phi=PHI_OBSERVATEUR,
    v_0=V_OBS,
    sigma=sigma,
    x_init=x_emetteur_init,
)

In [ ]:
inv_fisher = np.linalg.inv(fisher)

x_error = np.sqrt(inv_fisher[:, 0, 0])
y_error = np.sqrt(inv_fisher[:, 1, 1])
vx_error = np.sqrt(inv_fisher[:, 2, 2])
vy_error = np.sqrt(inv_fisher[:, 3, 3])
time = DELTA_T * np.array(range(100))

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.plot(time, x_error, color="blue", label="$x$ error")
ax.plot(time, y_error, color="red", label="$y$ error")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.plot(time, vx_error, color="blue", label="$v_x$ error")
ax.plot(time, vy_error, color="red", label="$v_y$ error")
plt.show()

In [ ]:
def optimizing_phi(**kwargs):
    test_values = np.linspace(start=0, stop=np.pi, num=100)
    res = np.empty(shape=(100,))
    for j, phi in enumerate(test_values):
        fishers = update_fisher(phi=phi, **kwargs)
        end_fisher_inv = np.linalg.inv(fishers[-1])
        res[j] = np.sqrt(end_fisher_inv[0, 0] + end_fisher_inv[1, 1])
    return test_values, res


# A rendre le 10 novembre

In [ ]:
phis, error_of_phi = optimizing_phi(
    p_0=p_0,
    delta_t=DELTA_T,
    n_before_turn=N_BEFORE_TURN,
    n_tot=N_TOT,
    v_0=V_OBS,
    sigma=sigma,
    x_init=x_emetteur_init,
)

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.plot(phis, error_of_phi, color="blue", label="$\\sqrt{\\sigma_x^2 + \\sigma_y^2}$")
plt.show()